<a href="https://colab.research.google.com/github/satani99/generative_deep_learning/blob/main/03_variational_autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn 
import torch

In [45]:
# The Encoder
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()

    self.conv0 = nn.Conv2d(1, 32, kernel_size=3, stride=2)
    self.leaky_relu1 = nn.LeakyReLU(32)
    self.conv1 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
    self.leaky_relu2 = nn.LeakyReLU(64)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu3 = nn.LeakyReLU(64)
    self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu4 = nn.LeakyReLU(64)
    self.linear = nn.Linear(256, 2)

  def forward(self, x):
    x = self.conv0(x)
    x = self.leaky_relu1(x)
    x = self.conv1(x)
    x = self.leaky_relu2(x)
    x = self.conv2(x)
    x = self.leaky_relu3(x)
    x = self.conv3(x)
    x = self.leaky_relu4(x)
    x = torch.flatten(x, 1)
    output = self.linear(x.view(-1, 256))
    return output
              

In [46]:
encoder = Encoder()
print(encoder)

Encoder(
  (conv0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu1): LeakyReLU(negative_slope=32)
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu2): LeakyReLU(negative_slope=64)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu3): LeakyReLU(negative_slope=64)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu4): LeakyReLU(negative_slope=64)
  (linear): Linear(in_features=256, out_features=2, bias=True)
)


In [76]:
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()

    self.linear = nn.Linear(2, 256)
    self.conv_t_0 = nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu5 = nn.LeakyReLU(64)
    self.conv_t_1 = nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1)
    self.leaky_relu6 = nn.LeakyReLU(64)
    self.conv_t_2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2)
    self.leaky_relu7 = nn.LeakyReLU(32)
    self.conv_t_3 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, output_padding=1)
    self.leaky_relu8 = nn.LeakyReLU(1)

  def forward(self, x):
    x = self.linear(x)
    x = self.leaky_relu5(self.conv_t_0(x.view(32, 64, 2, 2)))
    x = self.leaky_relu6(self.conv_t_1(x))
    x = self.leaky_relu7(self.conv_t_2(x))
    x = self.leaky_relu8(self.conv_t_3(x))
    output = torch.sigmoid(x)
    return output
    

In [77]:
decoder = Decoder()
print(decoder)

Decoder(
  (linear): Linear(in_features=2, out_features=256, bias=True)
  (conv_t_0): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu5): LeakyReLU(negative_slope=64)
  (conv_t_1): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (leaky_relu6): LeakyReLU(negative_slope=64)
  (conv_t_2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
  (leaky_relu7): LeakyReLU(negative_slope=32)
  (conv_t_3): ConvTranspose2d(32, 1, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
  (leaky_relu8): LeakyReLU(negative_slope=1)
)


In [71]:
tensor_transform = transforms.ToTensor()

dataset = datasets.MNIST(root = "./data",
                         train = True,
                         download = True,
                         transform = tensor_transform)

loader = torch.utils.data.DataLoader(dataset = dataset,
                                     batch_size = 32,
                                     shuffle = True)

In [78]:
class AE(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = Encoder()
    self.decoder = Decoder()

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [79]:
model = AE()

loss_function = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-1,
                             weight_decay = 1e-8)

In [81]:
epochs = 20
losses = []

for epoch in range(epochs):
  for (image, _) in loader:

    # image = image.reshape(1, -1, 28, 28)
    reconstructed = model(image)
    # print(image.shape)
    # print(reconstructed.shape)
    loss = loss_function(reconstructed, image)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss)

plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')

plt.plot(losses[-100:])


KeyboardInterrupt: ignored